In [1]:
import pyarrow
import pandas as pd
from pathlib import Path
import sys

ROOT = Path.cwd().resolve().parents[2]
print(ROOT)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
from app.src.utils.metrics import cosine_sim, text_cosine_sim

pd.set_option("display.max_colwidth", 120)  # don’t truncate cell text

/Users/jaelinlee/Documents/GitHub/Driftville_Agent


In [11]:
df = pd.read_parquet(ROOT / 'app/logs/v5/session_orpda_20251205_195057.log.parquet')
df.head()

,ts_created,tick,sim_time,agent,use_drift,orpda.observation.datetime_start,orpda.observation.duration_min,orpda.observation.location,orpda.observation.action,orpda.observation.state_summary,...,orpda.action_result.duration_min,orpda.action_result.location,orpda.action_result.action,orpda.action_result.drift_type,orpda.action_result.topic,orpda.action_result.state_summary,orpda.action_result.drift_topic,orpda.action_result.next_datetime,orpda.observation.notes,source_log
0,2025-12-05T19:51:01.839561-05:00,0,2023-02-13 06:00,Mei Lin,True,2023-02-13 06:00,15,home:bathroom,morning_routine,"Waking up and starting the morning routine in the bathroom, with sounds of the family and phone notifications.",...,15,home:kitchen,eating,internal,Breakfast with Eddy and thoughts about his schoolwork,"Eating breakfast with Eddy, mind wandering to his schoolwork and future.",Thoughts about Eddy's schoolwork and future,2023-02-13 06:15,None,app/logs/session_orpda_20251205_195057.log
1,2025-12-05T19:51:05.912323-05:00,1,2023-02-13 06:15,Mei Lin,True,2023-02-13 06:15,15,home:kitchen,eating,"Continuing breakfast with Eddy, discussing his schoolwork and planning the day ahead.",...,15,home:kitchen,eating,internal,Reflecting on Eddy's future career path,Finishing breakfast and internally reflecting on Eddy's potential career paths.,Reflecting on Eddy's future career path based on his current interests,2023-02-13 06:30,None,app/logs/session_orpda_20251205_195057.log
2,2025-12-05T19:51:10.154043-05:00,2,2023-02-13 06:30,Mei Lin,True,2023-02-13 06:30,15,home:kitchen,eating,Finishing breakfast with Eddy and preparing for the day ahead.,...,15,home:bathroom,morning_routine,internal,Preparing for teaching,Finishing breakfast and transitioning thoughts to the upcoming teaching session.,Next task: preparing for teaching,2023-02-13 06:45,None,app/logs/session_orpda_20251205_195057.log
3,2025-12-05T19:51:14.954279-05:00,3,2023-02-13 06:45,Mei Lin,True,2023-02-13 06:45,15,home:bedroom,morning_routine,Getting dressed and preparing for the day after her morning routine.,...,15,home:bedroom,morning_routine,internal,Thinking about Eddy's school play,"Getting dressed, with thoughts drifting to Eddy's school play and a recent conversation.",Thinking about her son Eddy's upcoming school play,2023-02-13 07:00,None,app/logs/session_orpda_20251205_195057.log
4,2025-12-05T19:51:19.407048-05:00,4,2023-02-13 07:00,Mei Lin,True,2023-02-13 07:00,15,home:bathroom,morning_routine,"Starting the morning routine in the bathroom, with sounds of the family waking and phone notifications.",...,15,home:kitchen,eating,internal,Next class preparation,Performing morning routine while thinking about lecture notes and student questions for the next class.,Next class preparation,2023-02-13 07:15,None,app/logs/session_orpda_20251205_195057.log


## Add Cosine Similarity

Planned vs. Action:
1. topics
2. summary
3. location

Next Steps:
1. embedd texts
2. text_cosine_sim
3. add to a new column

In [12]:
topic_cols=['orpda.plan.topic', 'orpda.action_result.topic']
summary_cols=['orpda.plan.state_summary', 'orpda.action_result.state_summary']

In [13]:
# 1. Add action similarity
df["topic_sim"] = df.apply(
    lambda r: text_cosine_sim(r[topic_cols[0]], r[topic_cols[1]]),
    axis=1,
)

# 2. Add summary similarity
df["action_sim"] = df.apply(
    lambda r: text_cosine_sim(r[summary_cols[0]], r[summary_cols[1]]),
    axis=1,
)
df.head()

,ts_created,tick,sim_time,agent,use_drift,orpda.observation.datetime_start,orpda.observation.duration_min,orpda.observation.location,orpda.observation.action,orpda.observation.state_summary,...,orpda.action_result.action,orpda.action_result.drift_type,orpda.action_result.topic,orpda.action_result.state_summary,orpda.action_result.drift_topic,orpda.action_result.next_datetime,orpda.observation.notes,source_log,topic_sim,action_sim
0,2025-12-05T19:51:01.839561-05:00,0,2023-02-13 06:00,Mei Lin,True,2023-02-13 06:00,15,home:bathroom,morning_routine,"Waking up and starting the morning routine in the bathroom, with sounds of the family and phone notifications.",...,eating,internal,Breakfast with Eddy and thoughts about his schoolwork,"Eating breakfast with Eddy, mind wandering to his schoolwork and future.",Thoughts about Eddy's schoolwork and future,2023-02-13 06:15,None,app/logs/session_orpda_20251205_195057.log,0.791282,0.763453
1,2025-12-05T19:51:05.912323-05:00,1,2023-02-13 06:15,Mei Lin,True,2023-02-13 06:15,15,home:kitchen,eating,"Continuing breakfast with Eddy, discussing his schoolwork and planning the day ahead.",...,eating,internal,Reflecting on Eddy's future career path,Finishing breakfast and internally reflecting on Eddy's potential career paths.,Reflecting on Eddy's future career path based on his current interests,2023-02-13 06:30,None,app/logs/session_orpda_20251205_195057.log,0.363649,0.741797
2,2025-12-05T19:51:10.154043-05:00,2,2023-02-13 06:30,Mei Lin,True,2023-02-13 06:30,15,home:kitchen,eating,Finishing breakfast with Eddy and preparing for the day ahead.,...,morning_routine,internal,Preparing for teaching,Finishing breakfast and transitioning thoughts to the upcoming teaching session.,Next task: preparing for teaching,2023-02-13 06:45,None,app/logs/session_orpda_20251205_195057.log,0.382423,0.570414
3,2025-12-05T19:51:14.954279-05:00,3,2023-02-13 06:45,Mei Lin,True,2023-02-13 06:45,15,home:bedroom,morning_routine,Getting dressed and preparing for the day after her morning routine.,...,morning_routine,internal,Thinking about Eddy's school play,"Getting dressed, with thoughts drifting to Eddy's school play and a recent conversation.",Thinking about her son Eddy's upcoming school play,2023-02-13 07:00,None,app/logs/session_orpda_20251205_195057.log,0.359638,0.534781
4,2025-12-05T19:51:19.407048-05:00,4,2023-02-13 07:00,Mei Lin,True,2023-02-13 07:00,15,home:bathroom,morning_routine,"Starting the morning routine in the bathroom, with sounds of the family waking and phone notifications.",...,eating,internal,Next class preparation,Performing morning routine while thinking about lecture notes and student questions for the next class.,Next class preparation,2023-02-13 07:15,None,app/logs/session_orpda_20251205_195057.log,0.316875,0.537014


## Save updated df with similarity to CSV

In [22]:
# df.to_parquet(ROOT / 'app/logs/combined_sessions_with_similarity.parquet', index=False)
df.to_csv(ROOT / 'app/logs/v5/combined_sessions_with_similarity.csv', index=False)

## Load from CSV

In [23]:
df = pd.read_csv(ROOT / 'app/logs/v5/combined_sessions_with_similarity.csv')
df['topic_sim'] = round(df['topic_sim'], 2)
df['action_sim'] = round(df['action_sim'], 2)
df.sample(5)

,ts_created,tick,sim_time,agent,use_drift,orpda.observation.datetime_start,orpda.observation.duration_min,orpda.observation.location,orpda.observation.action,orpda.observation.state_summary,...,orpda.action_result.action,orpda.action_result.drift_type,orpda.action_result.topic,orpda.action_result.state_summary,orpda.action_result.drift_topic,orpda.action_result.next_datetime,orpda.observation.notes,source_log,topic_sim,action_sim
6,2025-12-05T19:51:28.520186-05:00,6,2023-02-13 07:30,Mei Lin,True,2023-02-13 07:30,15,home:kitchen,eating,Mei is eating breakfast and discussing Eddy's schoolwork with him.,...,eating,internal,Research paper ideas,"Eating breakfast, mind drifts to research paper ideas while briefly discussing Eddy's schoolwork.",Research paper ideas,2023-02-13 07:45,NaN,app/logs/session_orpda_20251205_195057.log,0.32,0.70
4,2025-12-05T19:51:19.407048-05:00,4,2023-02-13 07:00,Mei Lin,True,2023-02-13 07:00,15,home:bathroom,morning_routine,"Starting the morning routine in the bathroom, with sounds of the family waking and phone notifications.",...,eating,internal,Next class preparation,Performing morning routine while thinking about lecture notes and student questions for the next class.,Next class preparation,2023-02-13 07:15,NaN,app/logs/session_orpda_20251205_195057.log,0.32,0.54
7,2025-12-05T19:51:32.572691-05:00,7,2023-02-13 07:45,Mei Lin,True,2023-02-13 07:45,15,home:kitchen,eating,"Finishing breakfast, preparing to commute to Oak Hill College.",...,eating,internal,Research paper ideas,"Finishing breakfast, with thoughts lingering on research paper ideas before commuting.",Research paper ideas,2023-02-13 08:00,NaN,app/logs/session_orpda_20251205_195057.log,0.25,0.57
8,2025-12-05T19:51:36.785009-05:00,8,2023-02-13 08:00,Mei Lin,True,2023-02-13 08:00,15,home:kitchen,eating,"Mei is eating breakfast with her son, Eddy, discussing schoolwork and family matters.",...,eating,internal,Research paper ideas,"Mei is eating breakfast with Eddy, her thoughts drifting to her research paper ideas.",Research paper ideas,2023-02-13 08:15,NaN,app/logs/session_orpda_20251205_195057.log,0.28,0.69
1,2025-12-05T19:51:05.912323-05:00,1,2023-02-13 06:15,Mei Lin,True,2023-02-13 06:15,15,home:kitchen,eating,"Continuing breakfast with Eddy, discussing his schoolwork and planning the day ahead.",...,eating,internal,Reflecting on Eddy's future career path,Finishing breakfast and internally reflecting on Eddy's potential career paths.,Reflecting on Eddy's future career path based on his current interests,2023-02-13 06:30,NaN,app/logs/session_orpda_20251205_195057.log,0.36,0.74


In [19]:
df.filter(like="plan").head(2)

,orpda.reflection.plan_alignment,orpda.plan.location,orpda.plan.action,orpda.plan.datetime_start,orpda.plan.duration_min,orpda.plan.topic,orpda.plan.state_summary
0,aligned,home:kitchen,eating,2023-02-13 06:00,15,Breakfast with Eddy,"Having breakfast with her son, Eddy, in the kitchen."
1,aligned,home:kitchen,eating,2023-02-13 06:15,15,Finishing breakfast and transitioning to morning routine,Finishing breakfast with Eddy and preparing to start her morning routine.


## Planned action topic vs. Actual action topic

In [21]:
base_cols = ['sim_time', 'agent', 'use_drift', ]
sim_cols = ['topic_sim', 'action_sim']
location_cols = ['orpda.observation.location', 'orpda.reflection.reasoning', 'orpda.plan.location', 'orpda.reflection.plan_alignment', 'orpda.action_result.location']

summary_df = df[base_cols + location_cols[:] + topic_cols + [sim_cols[0]] + summary_cols + [sim_cols[1]]]
summary_df[(summary_df.use_drift==True) & (summary_df.agent=='Mei Lin')]

,sim_time,agent,use_drift,orpda.observation.location,orpda.reflection.reasoning,orpda.plan.location,orpda.reflection.plan_alignment,orpda.action_result.location,orpda.plan.topic,orpda.action_result.topic,topic_sim,orpda.plan.state_summary,orpda.action_result.state_summary,action_sim
0,2023-02-13 06:00,Mei Lin,True,home:bathroom,"Mei is following her schedule. No topics have repeated, and there are no signs of fatigue or attention drift.",home:kitchen,aligned,home:kitchen,Breakfast with Eddy,Breakfast with Eddy and thoughts about his schoolwork,0.79,"Having breakfast with her son, Eddy, in the kitchen.","Eating breakfast with Eddy, mind wandering to his schoolwork and future.",0.76
1,2023-02-13 06:15,Mei Lin,True,home:kitchen,"Mei Lin is on track with her schedule. The topic of Eddy's schoolwork is not repetitive, and her attention is stable.",home:kitchen,aligned,home:kitchen,Finishing breakfast and transitioning to morning routine,Reflecting on Eddy's future career path,0.36,Finishing breakfast with Eddy and preparing to start her morning routine.,Finishing breakfast and internally reflecting on Eddy's potential career paths.,0.74
2,2023-02-13 06:30,Mei Lin,True,home:kitchen,Mei is on track with her schedule and shows no signs of fatigue or fixation.,home:bathroom,aligned,home:bathroom,Morning routine,Preparing for teaching,0.38,Starting the morning routine in the bathroom to prepare for the day.,Finishing breakfast and transitioning thoughts to the upcoming teaching session.,0.57
3,2023-02-13 06:45,Mei Lin,True,home:bedroom,Mei Lin is following her schedule and recent history shows no repeated topics or signs of fatigue. Attention is stable.,home:bedroom,aligned,home:bedroom,Getting dressed and organizing for the day,Thinking about Eddy's school play,0.36,Mei is getting dressed and organizing her belongings in her bedroom.,"Getting dressed, with thoughts drifting to Eddy's school play and a recent conversation.",0.53
4,2023-02-13 07:00,Mei Lin,True,home:bathroom,Mei Lin is currently following her schedule. Her attention is stable and there are no repeating topics.,home:kitchen,aligned,home:kitchen,Breakfast with Eddy,Next class preparation,0.32,"Having breakfast with her son, Eddy, in the kitchen.",Performing morning routine while thinking about lecture notes and student questions for the next class.,0.54
5,2023-02-13 07:15,Mei Lin,True,home:kitchen,Mei Lin is actively engaged in her scheduled activities with no significant topic repetition or signs of fatigue. At...,home:kitchen,aligned,home:kitchen,Finishing breakfast and preparing for commute,Research paper ideas,0.20,Finishing breakfast with Eddy and preparing mentally for the commute to Oak Hill College.,"Eating breakfast, mind drifting to research paper ideas.",0.53
6,2023-02-13 07:30,Mei Lin,True,home:kitchen,Mei is on track. No significant drift or fatigue detected. Topic repetition is low.,home:kitchen,aligned,home:kitchen,Discussing Eddy's schoolwork,Research paper ideas,0.32,Continue eating breakfast and discussing Eddy's schoolwork to maintain family connection.,"Eating breakfast, mind drifts to research paper ideas while briefly discussing Eddy's schoolwork.",0.70
7,2023-02-13 07:45,Mei Lin,True,home:kitchen,"Mei Lin is following her schedule. 'Research paper ideas' appeared twice, but she is transitioning. Fatigue is low.",home:kitchen,aligned,home:kitchen,Finishing breakfast,Research paper ideas,0.25,Mei is finishing breakfast and preparing for her commute to Oak Hill College.,"Finishing breakfast, with thoughts lingering on research paper ideas before commuting.",0.57
8,2023-02-13 08:00,Mei Lin,True,home:kitchen,"Mei is engaged in scheduled family breakfast. 'Research paper ideas' has appeared twice recently, hence the count. F...",home:kitchen,aligned,home:kitchen,Family breakfast and Eddy's schoolwork,Research paper ideas,0.28,"Continue eating breakfast with Eddy, discussing his schoolwork and any family matters.","Mei is eating breakfast with Eddy, her tho

**Observation:**
- observed location != (Planned location == Action location) -- is this wrong? why are they different? isn't observed location current scheduled location?

**Answer:**

- No. It’s expected that they can differ. In the logs:
    - **observation.location** = where the simulated agent actually is at that tick (state fed to the loop).
    - **plan.location** = the scheduled/intent location for the action at that tick.
    - **action_result.location** = where the agent ended up acting after applying any drift/decisions.
- So we can see observation sometimes reflects the prior step (e.g., still in bathroom) while plan is the next scheduled move (e.g., breakfast in kitchen). 
- If drift nudges behavior, action_result can diverge from plan too.
- When **drift_type** is **none, internal, or attentional_leak**, action_result.location and action are now forced to match the plan (and topic defaults to the plan if missing). 
- This makes observation in the next tick align with the planned location for non-behavioral cases, avoiding the kitchen/bathroom mismatch we saw.